<a href="https://colab.research.google.com/github/yeon524/IDS/blob/main/IDS_obj4_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

base_url = "https://zenodo.org/record/8145223/files"

files = [
    "preprocessed-IMU-with-operation-labels.zip",
    "U0101.zip","U0102.zip","U0103.zip","U0104.zip","U0105.zip",
    "U0106.zip","U0107.zip","U0108.zip","U0109.zip","U0110.zip","U0111.zip",
    "U0201.zip","U0202.zip","U0203.zip","U0204.zip","U0205.zip",
    "U0206.zip","U0207.zip","U0208.zip","U0209.zip","U0210.zip"
]

os.makedirs("openpack_raw", exist_ok=True)

for f in files:
    url = f"{base_url}/{f}?download=1"
    save_path = f"openpack_raw/{f}"

    print(f"⬇️ Downloading {f} ...")
    !wget -q "{url}" -O "{save_path}"
    print(f"   ✔ Saved to {save_path}")

print("\n🎉 다운로드 완료!")

os.makedirs("openpack", exist_ok=True)

for f in files:
    zip_path = f"openpack_raw/{f}"
    if f.endswith(".zip"):
        extract_dir = f"openpack/{f.replace('.zip','')}"
        print(f"📦 Extracting {f} ...")
        !unzip -q "{zip_path}" -d "{extract_dir}"

print("\n🎉 모든 파일 다운로드 + 압축 해제 완료!")


⬇️ Downloading preprocessed-IMU-with-operation-labels.zip ...
   ✔ Saved to openpack_raw/preprocessed-IMU-with-operation-labels.zip
⬇️ Downloading U0101.zip ...
   ✔ Saved to openpack_raw/U0101.zip
⬇️ Downloading U0102.zip ...
   ✔ Saved to openpack_raw/U0102.zip
⬇️ Downloading U0103.zip ...
   ✔ Saved to openpack_raw/U0103.zip
⬇️ Downloading U0104.zip ...
   ✔ Saved to openpack_raw/U0104.zip
⬇️ Downloading U0105.zip ...
   ✔ Saved to openpack_raw/U0105.zip
⬇️ Downloading U0106.zip ...
   ✔ Saved to openpack_raw/U0106.zip
⬇️ Downloading U0107.zip ...
   ✔ Saved to openpack_raw/U0107.zip
⬇️ Downloading U0108.zip ...
   ✔ Saved to openpack_raw/U0108.zip
⬇️ Downloading U0109.zip ...
   ✔ Saved to openpack_raw/U0109.zip
⬇️ Downloading U0110.zip ...
   ✔ Saved to openpack_raw/U0110.zip
⬇️ Downloading U0111.zip ...
   ✔ Saved to openpack_raw/U0111.zip
⬇️ Downloading U0201.zip ...
   ✔ Saved to openpack_raw/U0201.zip
⬇️ Downloading U0202.zip ...
   ✔ Saved to openpack_raw/U0202.zip
⬇️ Downloa

In [2]:
!rm -rf openpack-dataset
!git clone https://github.com/yeon524/openpack-dataset.git -q

In [3]:
import os
import pandas as pd
import numpy as np
from glob import glob

BASE = "openpack"

all_rows = []

# openpack/Uxxxx/annotation/openpack-actions/S*.csv
action_files = glob(os.path.join(BASE, "U0*", "annotation", "openpack-actions", "S*.csv"))
print("찾은 action csv 개수:", len(action_files))

for fpath in action_files:
    parts = fpath.split(os.sep)
    subject_id = parts[-4]
    session_id = os.path.splitext(parts[-1])[0]

    df = pd.read_csv(fpath)

    if not {"start", "end"}.issubset(df.columns):
        continue

    op_col = None
    for cand in ["operation", "action", "op", "op_id", "label"]:
        if cand in df.columns:
            op_col = cand
            break
    if op_col is None:
        continue

    df["Subject"] = subject_id
    df["Session"] = session_id
    df["operation_name"] = df[op_col].astype(str)

    df["start_dt"] = pd.to_datetime(df["start"], errors="coerce")
    df["end_dt"]   = pd.to_datetime(df["end"], errors="coerce")
    df["duration_sec"] = (df["end_dt"] - df["start_dt"]).dt.total_seconds()

    all_rows.append(df[["Subject", "Session", "operation_name", "duration_sec"]])

if not all_rows:
    raise ValueError("raw_ops를 못 만들었어. 경로/컬럼명을 확인해야 해!")

raw_ops = pd.concat(all_rows, ignore_index=True)

raw_ops["duration_sec"] = pd.to_numeric(raw_ops["duration_sec"], errors="coerce")
raw_ops = raw_ops.dropna(subset=["duration_sec"])
raw_ops = raw_ops[raw_ops["duration_sec"] > 0]

raw_ops["Scenario"] = raw_ops["Session"].str.extract(r"(S\d)").iloc[:, 0]

print("raw_ops shape:", raw_ops.shape)
print(raw_ops.head())
print("\duration_sec 요약")
print(raw_ops["duration_sec"].describe())

<>:55: SyntaxWarning: invalid escape sequence '\d'
<>:55: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2637106538.py:55: SyntaxWarning: invalid escape sequence '\d'
  print("\duration_sec 요약")


찾은 action csv 개수: 104
raw_ops shape: (53081, 5)
  Subject Session       operation_name  duration_sec Scenario
0   U0110   S0500              Picking         1.785       S0
1   U0110   S0500              Picking         2.320       S0
2   U0110   S0500  Relocate Item Label         1.222       S0
3   U0110   S0500  Relocate Item Label         2.389       S0
4   U0110   S0500         Assemble Box         2.174       S0
\duration_sec 요약
count    53081.000000
mean         2.323439
std          2.596442
min          0.083000
25%          0.744000
50%          1.508000
75%          3.096000
max        124.181000
Name: duration_sec, dtype: float64


In [4]:
import re
import pandas as pd
import numpy as np

subjects_md_path = "/content/openpack-dataset/docs/data-collection/subjects.md"

rows = []
with open(subjects_md_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if "|" not in line:
            continue
        if "Subject" in line or "---" in line:
            continue

        parts = [c.strip() for c in line.split("|") if c.strip()]
        if not parts:
            continue
        if not parts[0].startswith("U0"):
            continue
        rows.append(parts[:6])

subjects_df = pd.DataFrame(
    rows,
    columns=["Subject", "Sex", "Age", "DominantHand", "Experience", "Note"]
)

def parse_age(age_str):
    if pd.isna(age_str):
        return np.nan
    m = re.match(r"(\d+)", str(age_str).strip())
    return int(m.group(1)) if m else np.nan

def parse_experience(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in ["", "-", "none"]:
        return 0

    m1 = re.match(r"(\d+)\s*(month|months|year|years)", s)
    if m1:
        num = int(m1.group(1))
        unit = m1.group(2)
        return num * 12 if "year" in unit else num

    m2 = re.match(r"(\d+)\s*(m|y)", s)
    if m2:
        num = int(m2.group(1))
        unit = m2.group(2)

In [14]:
# ======================================================
# OpenPack Duration Prediction - Final Version (NO Feature Importance)
# ======================================================

import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

import seaborn as sns
sns.set(style="whitegrid")

# ======================================================
# 0. Prepare duration_sec, operation_name, Scenario
# ======================================================

if "duration_sec" not in raw_ops.columns and {"start", "end"}.issubset(raw_ops.columns):
    raw_ops["start_dt"] = pd.to_datetime(raw_ops["start"], errors="coerce")
    raw_ops["end_dt"]   = pd.to_datetime(raw_ops["end"], errors="coerce")
    raw_ops["duration_sec"] = (raw_ops["end_dt"] - raw_ops["start_dt"]).dt.total_seconds()

if "operation_name" not in raw_ops.columns and "operation" in raw_ops.columns:
    raw_ops["operation_name"] = raw_ops["operation"].astype(str)

if "Scenario" not in raw_ops.columns and "Session" in raw_ops.columns:
    raw_ops["Scenario"] = raw_ops["Session"].str.extract(r"(S\d)").iloc[:, 0]

# ======================================================
# 1. Create Age_num and Exp_months from subjects_df
# ======================================================

def parse_age(age_str):
    if pd.isna(age_str):
        return np.nan
    m = re.match(r"(\d+)", str(age_str))
    return int(m.group(1)) if m else np.nan

def parse_experience(exp):
    if pd.isna(exp):
        return np.nan
    s = str(exp).lower().strip()

    if s in ["", "-", "none"]:
        return 0

    m = re.match(r"(\d+)\s*(month|months|year|years|m|y)", s)
    if m:
        num = int(m.group(1))
        unit = m.group(2)
        return num * 12 if unit.startswith("y") else num

    if s.isdigit():
        return int(s)

    return np.nan

subjects_df["Age_num"] = subjects_df["Age"].apply(parse_age)
subjects_df["Exp_months"] = subjects_df["Experience"].apply(parse_experience)

# ======================================================
# 2. Clean duration and merge data
# ======================================================

ops_clean = raw_ops.copy()
ops_clean["duration_sec"] = pd.to_numeric(ops_clean["duration_sec"], errors="coerce")
ops_clean = ops_clean.dropna(subset=["duration_sec"])
ops_clean = ops_clean[ops_clean["duration_sec"] > 0]

df = ops_clean.merge(subjects_df, on="Subject", how="left")

use_cols = [
    "duration_sec",
    "operation_name",
    "Scenario",
    "Age_num",
    "Exp_months",
    "Sex",
    "DominantHand"
]

df = df[use_cols].dropna(subset=["Age_num", "Exp_months"])

print("Modeling Data Shape:", df.shape)
print(df.head())

# ======================================================
# 3. Split X and y
# ======================================================

X = df.drop("duration_sec", axis=1)
y = df["duration_sec"]

# ======================================================
# 4. Train / Test Split
# ======================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ======================================================
# 5. Preprocessing and model definition
# ======================================================

categorical_cols = [c for c in ["operation_name", "Scenario", "Sex", "DominantHand"] if c in X.columns]
numeric_cols = ["Age_num", "Exp_months"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)

rf = RandomForestRegressor(
    n_estimators=400,
    random_state=42,
    n_jobs=-1
)

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("rf", rf)
])

# ======================================================
# 6. Model training
# ======================================================

model.fit(X_train, y_train)

# ======================================================
# 7. Model evaluation
# ======================================================

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = model.predict(X_test)

mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print("\n=== Duration Prediction Performance ===")
print(f"MSE : {mse:.3f} (seconds^2)")
print(f"RMSE: {rmse:.3f} seconds")
print(f"MAE : {mae:.3f} seconds")
print(f"R²  : {r2:.3f}")

# ======================================================
# 8. Prediction function
# ======================================================

def predict_future_duration(
    operation_name,
    age,
    exp_months,
    sex="F",
    dominant_hand="Right",
    scenario="S1"
):
    sample = {
        "operation_name": operation_name,
        "Scenario": scenario,
        "Age_num": age,
        "Exp_months": exp_months,
        "Sex": sex,
        "DominantHand": dominant_hand
    }
    sample_df = pd.DataFrame([sample])
    return model.predict(sample_df)[0]

# ======================================================
# 9. User input based prediction
# ======================================================

print("\n=== Duration Prediction Model (Random Forest) ===")
print("Please enter the values in order.\n")

age = int(input("Age (Age_num) (ex: 30): "))
exp_months = int(input("Experience in months (Exp_months) (ex: 24): "))

sex = input("Sex (M/F): ").strip().upper()
if sex not in ["M", "F"]:
    sex = "F"

dominant_hand = input("Dominant hand (Right/Left): ").strip().title()
if dominant_hand not in ["Right", "Left"]:
    dominant_hand = "Right"

scenario = input("Scenario (S1–S4): ").strip().upper()
operation_name = input("Operation name (ex: Close Box): ").strip()

pred = predict_future_duration(
    operation_name=operation_name,
    age=age,
    exp_months=exp_months,
    sex=sex,
    dominant_hand=dominant_hand,
    scenario=scenario
)

print(f"\n Predicted task duration: {pred:.2f} seconds")

Modeling Data Shape: (48040, 7)
   duration_sec       operation_name Scenario  Age_num  Exp_months Sex  \
0         1.785              Picking       S0     40.0          10   F   
1         2.320              Picking       S0     40.0          10   F   
2         1.222  Relocate Item Label       S0     40.0          10   F   
3         2.389  Relocate Item Label       S0     40.0          10   F   
4         2.174         Assemble Box       S0     40.0          10   F   

  DominantHand  
0        Right  
1        Right  
2        Right  
3        Right  
4        Right  

=== Duration Prediction Performance ===
MSE : 4.267 (seconds^2)
RMSE: 2.066 seconds
MAE : 1.340 seconds
R²  : 0.214

=== Duration Prediction Model (Random Forest) ===
Please enter the values in order.

Age (Age_num) (ex: 30): 30
Experience in months (Exp_months) (ex: 24): 24
Sex (M/F): F
Dominant hand (Right/Left): Right
Scenario (S1–S4): S3
Operation name (ex: Close Box): Close Box

 Predicted task duration: 3.61 se

In [16]:
# ======================================================
# OpenPack Duration Prediction - XGBoost Final Version
# (NO Feature Importance, English prompts, Input-based, Metrics)
# ======================================================

import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import seaborn as sns
sns.set(style="whitegrid")

# ✅ 1) Install XGBoost if needed (Colab: run once)
# !pip install xgboost

from xgboost import XGBRegressor

# ======================================================
# 0. Prepare duration_sec, operation_name, Scenario
# ======================================================

if "duration_sec" not in raw_ops.columns and {"start", "end"}.issubset(raw_ops.columns):
    raw_ops["start_dt"] = pd.to_datetime(raw_ops["start"], errors="coerce")
    raw_ops["end_dt"]   = pd.to_datetime(raw_ops["end"], errors="coerce")
    raw_ops["duration_sec"] = (raw_ops["end_dt"] - raw_ops["start_dt"]).dt.total_seconds()

if "operation_name" not in raw_ops.columns and "operation" in raw_ops.columns:
    raw_ops["operation_name"] = raw_ops["operation"].astype(str)

if "Scenario" not in raw_ops.columns and "Session" in raw_ops.columns:
    raw_ops["Scenario"] = raw_ops["Session"].str.extract(r"(S\d)").iloc[:, 0]

# ======================================================
# 1. Create Age_num and Exp_months from subjects_df
# ======================================================

def parse_age(age_str):
    if pd.isna(age_str):
        return np.nan
    m = re.match(r"(\d+)", str(age_str))
    return int(m.group(1)) if m else np.nan

def parse_experience(exp):
    if pd.isna(exp):
        return np.nan
    s = str(exp).lower().strip()

    if s in ["", "-", "none"]:
        return 0

    m = re.match(r"(\d+)\s*(month|months|year|years|m|y)", s)
    if m:
        num = int(m.group(1))
        unit = m.group(2)
        return num * 12 if unit.startswith("y") else num

    if s.isdigit():
        return int(s)

    return np.nan

subjects_df["Age_num"] = subjects_df["Age"].apply(parse_age)
subjects_df["Exp_months"] = subjects_df["Experience"].apply(parse_experience)

# ======================================================
# 2. Clean duration and merge data
# ======================================================

ops_clean = raw_ops.copy()
ops_clean["duration_sec"] = pd.to_numeric(ops_clean["duration_sec"], errors="coerce")
ops_clean = ops_clean.dropna(subset=["duration_sec"])
ops_clean = ops_clean[ops_clean["duration_sec"] > 0]

df = ops_clean.merge(subjects_df, on="Subject", how="left")

use_cols = [
    "duration_sec",
    "operation_name",
    "Scenario",
    "Age_num",
    "Exp_months",
    "Sex",
    "DominantHand"
]

df = df[use_cols].dropna(subset=["Age_num", "Exp_months"])

print("Modeling Data Shape:", df.shape)
print(df.head())

# ======================================================
# 3. Split X and y
# ======================================================

X = df.drop("duration_sec", axis=1)
y = df["duration_sec"]

# ======================================================
# 4. Train / Test Split
# ======================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ======================================================
# 5. Preprocessing + XGBoost model
# ======================================================

categorical_cols = [c for c in ["operation_name", "Scenario", "Sex", "DominantHand"] if c in X.columns]
numeric_cols = ["Age_num", "Exp_months"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)

xgb = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("xgb", xgb)
])

# ======================================================
# 6. Train
# ======================================================

model.fit(X_train, y_train)

# ======================================================
# 7. Evaluation (MSE, RMSE, MAE, R^2)
# ======================================================

y_pred = model.predict(X_test)

mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print("\n=== Duration Prediction Performance (XGBoost) ===")
print(f"MSE : {mse:.3f} (seconds^2)")
print(f"RMSE: {rmse:.3f} seconds")
print(f"MAE : {mae:.3f} seconds")
print(f"R²  : {r2:.3f}")

# ======================================================
# 8. Prediction function
# ======================================================

def predict_future_duration_xgb(
    operation_name,
    age,
    exp_months,
    sex="F",
    dominant_hand="Right",
    scenario="S1"
):
    sample = {
        "operation_name": operation_name,
        "Scenario": scenario,
        "Age_num": age,
        "Exp_months": exp_months,
        "Sex": sex,
        "DominantHand": dominant_hand
    }
    sample_df = pd.DataFrame([sample])
    return model.predict(sample_df)[0]

# ======================================================
# 9. User input based prediction
# ======================================================

print("\n=== Duration Prediction Model (XGBoost) ===")
print("Please enter the values in order.\n")

age = int(input("Age (Age_num) (ex: 30): "))
exp_months = int(input("Experience in months (Exp_months) (ex: 24): "))

sex = input("Sex (M/F): ").strip().upper()
if sex not in ["M", "F"]:
    sex = "F"

dominant_hand = input("Dominant hand (Right/Left): ").strip().title()
if dominant_hand not in ["Right", "Left"]:
    dominant_hand = "Right"

scenario = input("Scenario (S1–S4): ").strip().upper()
operation_name = input("Operation name (ex: Close Box): ").strip()

pred = predict_future_duration_xgb(
    operation_name=operation_name,
    age=age,
    exp_months=exp_months,
    sex=sex,
    dominant_hand=dominant_hand,
    scenario=scenario
)

print(f"\n✅ Predicted task duration: {pred:.2f} seconds")

Modeling Data Shape: (48040, 7)
   duration_sec       operation_name Scenario  Age_num  Exp_months Sex  \
0         1.785              Picking       S0     40.0          10   F   
1         2.320              Picking       S0     40.0          10   F   
2         1.222  Relocate Item Label       S0     40.0          10   F   
3         2.389  Relocate Item Label       S0     40.0          10   F   
4         2.174         Assemble Box       S0     40.0          10   F   

  DominantHand  
0        Right  
1        Right  
2        Right  
3        Right  
4        Right  

=== Duration Prediction Performance (XGBoost) ===
MSE : 4.261 (seconds^2)
RMSE: 2.064 seconds
MAE : 1.340 seconds
R²  : 0.215

=== Duration Prediction Model (XGBoost) ===
Please enter the values in order.

Age (Age_num) (ex: 30): 30
Experience in months (Exp_months) (ex: 24): 24
Sex (M/F): F
Dominant hand (Right/Left): Right
Scenario (S1–S4): S3
Operation name (ex: Close Box): Close Box

✅ Predicted task duration: 3.

In [19]:
# ======================================================
# OpenPack Duration Prediction - Linear Regression Final Version (Non-negative Prediction)
# (NO Feature Importance, English prompts, Input-based, Metrics)
# ======================================================

import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import seaborn as sns
sns.set(style="whitegrid")

# ======================================================
# 0. Prepare duration_sec, operation_name, Scenario
# ======================================================

if "duration_sec" not in raw_ops.columns and {"start", "end"}.issubset(raw_ops.columns):
    raw_ops["start_dt"] = pd.to_datetime(raw_ops["start"], errors="coerce")
    raw_ops["end_dt"]   = pd.to_datetime(raw_ops["end"], errors="coerce")
    raw_ops["duration_sec"] = (raw_ops["end_dt"] - raw_ops["start_dt"]).dt.total_seconds()

if "operation_name" not in raw_ops.columns and "operation" in raw_ops.columns:
    raw_ops["operation_name"] = raw_ops["operation"].astype(str)

if "Scenario" not in raw_ops.columns and "Session" in raw_ops.columns:
    raw_ops["Scenario"] = raw_ops["Session"].str.extract(r"(S\d)").iloc[:, 0]

# ======================================================
# 1. Create Age_num and Exp_months from subjects_df
# ======================================================

def parse_age(age_str):
    if pd.isna(age_str):
        return np.nan
    m = re.match(r"(\d+)", str(age_str))
    return int(m.group(1)) if m else np.nan

def parse_experience(exp):
    if pd.isna(exp):
        return np.nan
    s = str(exp).lower().strip()

    if s in ["", "-", "none"]:
        return 0

    m = re.match(r"(\d+)\s*(month|months|year|years|m|y)", s)
    if m:
        num = int(m.group(1))
        unit = m.group(2)
        return num * 12 if unit.startswith("y") else num

    if s.isdigit():
        return int(s)

    return np.nan

subjects_df["Age_num"] = subjects_df["Age"].apply(parse_age)
subjects_df["Exp_months"] = subjects_df["Experience"].apply(parse_experience)

# ======================================================
# 2. Clean duration and merge data
# ======================================================

ops_clean = raw_ops.copy()
ops_clean["duration_sec"] = pd.to_numeric(ops_clean["duration_sec"], errors="coerce")
ops_clean = ops_clean.dropna(subset=["duration_sec"])
ops_clean = ops_clean[ops_clean["duration_sec"] > 0]

df = ops_clean.merge(subjects_df, on="Subject", how="left")

use_cols = [
    "duration_sec",
    "operation_name",
    "Scenario",
    "Age_num",
    "Exp_months",
    "Sex",
    "DominantHand"
]

df = df[use_cols].dropna(subset=["Age_num", "Exp_months"])

print("Modeling Data Shape:", df.shape)
print(df.head())

# ======================================================
# 3. Split X and y
# ======================================================

X = df.drop("duration_sec", axis=1)
y = df["duration_sec"]

# ======================================================
# 4. Train / Test Split
# ======================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ======================================================
# 5. Preprocessing + Linear Regression model
# ======================================================

categorical_cols = [c for c in ["operation_name", "Scenario", "Sex", "DominantHand"] if c in X.columns]
numeric_cols = ["Age_num", "Exp_months"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)

lin = LinearRegression()

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("linreg", lin)
])

# ======================================================
# 6. Train
# ======================================================

model.fit(X_train, y_train)

# ======================================================
# 7. Evaluation (MSE, RMSE, MAE, R^2)
# ======================================================

y_pred = model.predict(X_test)

mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print("\n=== Duration Prediction Performance (Linear Regression) ===")
print(f"MSE : {mse:.3f} (seconds^2)")
print(f"RMSE: {rmse:.3f} seconds")
print(f"MAE : {mae:.3f} seconds")
print(f"R²  : {r2:.3f}")

# ======================================================
# 8. Prediction function (Non-negative)
# ======================================================

def predict_future_duration_lin(
    operation_name,
    age,
    exp_months,
    sex="F",
    dominant_hand="Right",
    scenario="S1"
):
    sample = {
        "operation_name": operation_name,
        "Scenario": scenario,
        "Age_num": age,
        "Exp_months": exp_months,
        "Sex": sex,
        "DominantHand": dominant_hand
    }
    sample_df = pd.DataFrame([sample])
    pred = model.predict(sample_df)[0]
    pred = max(pred, 0)  # ✅ prevent negative durations
    return pred

# ======================================================
# 9. User input based prediction
# ======================================================

print("\n=== Duration Prediction Model (Linear Regression) ===")
print("Please enter the values in order.\n")

age = int(input("Age (Age_num) (ex: 30): "))
exp_months = int(input("Experience in months (Exp_months) (ex: 24): "))

sex = input("Sex (M/F): ").strip().upper()
if sex not in ["M", "F"]:
    sex = "F"

dominant_hand = input("Dominant hand (Right/Left): ").strip().title()
if dominant_hand not in ["Right", "Left"]:
    dominant_hand = "Right"

scenario = input("Scenario (S1–S4): ").strip().upper()
operation_name = input("Operation name (ex: Close Box): ").strip()

pred = predict_future_duration_lin(
    operation_name=operation_name,
    age=age,
    exp_months=exp_months,
    sex=sex,
    dominant_hand=dominant_hand,
    scenario=scenario
)

print(f"\n✅ Predicted task duration: {pred:.2f} seconds")

Modeling Data Shape: (48040, 7)
   duration_sec       operation_name Scenario  Age_num  Exp_months Sex  \
0         1.785              Picking       S0     40.0          10   F   
1         2.320              Picking       S0     40.0          10   F   
2         1.222  Relocate Item Label       S0     40.0          10   F   
3         2.389  Relocate Item Label       S0     40.0          10   F   
4         2.174         Assemble Box       S0     40.0          10   F   

  DominantHand  
0        Right  
1        Right  
2        Right  
3        Right  
4        Right  

=== Duration Prediction Performance (Linear Regression) ===
MSE : 4.399 (seconds^2)
RMSE: 2.097 seconds
MAE : 1.365 seconds
R²  : 0.189

=== Duration Prediction Model (Linear Regression) ===
Please enter the values in order.

Age (Age_num) (ex: 30): 30
Experience in months (Exp_months) (ex: 24): 24
Sex (M/F): F
Dominant hand (Right/Left): Right
Scenario (S1–S4): S3
Operation name (ex: Close Box): Close Box

✅ Predict